In [1]:
import numpy as np
import pandas as pd
import scipy.sparse as sp

from dataset import UserSessionItemDataset
from evaluation import recall, ndcg, evaluate
from models import EASE, AbsEASE
from pipelines import hyperparameter_selection, run_test

from copy import deepcopy

np.random.seed(12345)

# preprocessing = extract away

In [2]:
DATA_FOLDER = 'data/'

df = pd.read_csv(DATA_FOLDER + 'processedRatingsBeer.csv')
df = df.drop(columns=['Unnamed: 0'])
df = df.rename(columns={"beerId": "item_id"})

df.head()

,index,brewery_id,brewery_name,timestamp,rating,review_aroma,review_appearance,userId,beer_style,review_palate,review_taste,beer_name,beer_abv,item_id,datetime,sessionId
0,614509,199,Ballast Point Brewing Company,1205561385,1.0,4.0,3.5,0110x011,American Double / Imperial IPA,4.0,4.0,Dorado Double IPA,9.6,10386,2008-03-15 06:09:45,1
1,182729,396,AleSmith Brewing Company,1205823873,1.0,5.0,4.0,0110x011,American Double / Imperial IPA,4.5,4.5,YuleSmith (Summer),8.5,7284,2008-03-18 07:04:33,2
2,1134674,863,Russian River Brewing Company,1207011338,1.0,5.0,5.0,0110x011,American Double / Imperial IPA,4.5,4.5,Pliny The Elder,8.0,7971,2008-04-01 00:55:38,3
3,462893,559,Speakeasy Ales & Lagers,1207362193,1.0,4.0,4.0,0110x011,American Double / Imperial IPA,3.5,4.0,Double Daddy Imperial India Pale Ale,9.5,25283,2008-04-05 02:23:13,4
4,886485,147,Stone Brewing Co.,1208211124,1.0,5.0,4.0,0110x011,American Black Ale,4.0,5.0,Stone Sublimely Self-Righteous Ale,8.7,38470,2008-04-14 22:12:04,5


In [3]:
df["sessionId"] = 1

In [4]:
# create a user_session identifier
df['user_session_id'] = (df['userId'].astype(str) + '_' + df['sessionId'].astype(str)).map(hash)
df = df.sort_values(by=['user_session_id', 'timestamp'])

In [5]:
testable_sessions = df.groupby('user_session_id').agg({"userId": "first", "rating": "count"}).rename(columns={"rating": "ratingsCount"}).reset_index()
testable_sessions["positiveCount"] = 0
positive_counts = df[df.rating == 1.].groupby('user_session_id').rating.count()
testable_sessions.loc[testable_sessions["user_session_id"].isin(positive_counts.index) ,"positiveCount"] = testable_sessions.loc[testable_sessions["user_session_id"].isin(positive_counts.index) ,"user_session_id"].apply(lambda x: positive_counts[x])
testable_sessions["negativeCount"] = testable_sessions["ratingsCount"] - testable_sessions["positiveCount"]

testable_sessions

,user_session_id,userId,ratingsCount,positiveCount,negativeCount
0,-9222769369198958281,nugget,2,2,0
1,-9222394865684918904,Danny2321,3,2,1
2,-9222300797394093464,CanonJohn,8,7,1
3,-9221777287235335041,groovy24,4,4,0
4,-9221714889734660647,khwheel,1,1,0
...,...,...,...,...,...
24569,9219640372787849508,larshultqvist,3,3,0
24570,9221344691897703657,russellgillette,2,2,0
24571,9221473476771149170,beertastegood,1,1,0
24572,9221758644131394282,Wiiare138,2,2,0


In [6]:
df.user_session_id.nunique(), df.item_id.nunique(), len(df)

(24574, 32148, 703891)

In [7]:
# testable_sessions = pd.read_csv(DATA_FOLDER + 'testableSessionsBeer.csv')

# testable_sessions.head()

In [8]:
len(testable_sessions)

24574

In [9]:
print(testable_sessions.positiveCount.min())
# we need at least 5 positive interactions and 5 negative interactions
testable_sessions["negativeCount"] = testable_sessions["ratingsCount"] - testable_sessions["positiveCount"]
testable_sessions = testable_sessions[(testable_sessions.positiveCount >= 3) & (testable_sessions.negativeCount >= 3)]
len(testable_sessions)

0


2517

In [10]:
# # create a user_session identifier
# testable_sessions['user_session_id'] = (testable_sessions['userId'].astype(str) + '_' + testable_sessions['sessionId'].astype(str)).map(hash)

In [11]:
# shuffle rows
testable_sessions = testable_sessions.sample(frac=1).reset_index(drop=True)
# add sessions randomly to validation set (0) or one of five CV splits (1-5)
testable_sessions["split"] = testable_sessions.index.values

split_number_to_string = {
    0: "val",  # 25% of sessions are validation
    1: "test",
    2: "test",
    3: "test",
}

testable_sessions["split"] = testable_sessions["split"].apply(lambda x: x % 4).apply(lambda x: split_number_to_string[x])

In [12]:
# check if every row is a unique user session
assert len(testable_sessions) == testable_sessions.user_session_id.nunique()

In [13]:
for split in ["train", "val", "test"]:
    print(f"number of testable sessions in split {split}: {len(testable_sessions[testable_sessions.split == split])}")

number of testable sessions in split train: 0
number of testable sessions in split val: 630
number of testable sessions in split test: 1887


In [14]:
# map user_session_id to split
user_session_id_to_split = {usid: split for usid, split in zip(testable_sessions.user_session_id.values, testable_sessions.split.values)}

In [15]:
# assign dataframe entries to split
df["split"] = df["user_session_id"].apply(lambda x: user_session_id_to_split.get(x, "train"))

In [16]:
train_df = df[df.split == "train"]
eval_df = df[df.split != "train"]

# to simulate real-world data sparsity, we subsample rows of train df in a stratified manner (per item)
# we modify the long-tail distribution to make it sparser
# set count x of every item to x**(1/1.5)
# max count will be 100, and the distribution will be long tailed
print(train_df.item_id.value_counts().describe())
train_df = train_df.groupby('item_id', group_keys=False).apply(
    lambda x: x.sample(
        min(
            round(len(x)**(2/3)), # at most this number
            100,  # at most 100
        ),
    )
)
print(train_df.item_id.value_counts().describe())
# in training split, keep only interactions with (positive) rating 1
train_df = train_df[train_df.rating == 1.]

df = pd.concat([train_df, eval_df])

count    17707.000000
mean        13.555938
std         47.063999
min          1.000000
25%          1.000000
50%          2.000000
75%          7.000000
max       1251.000000
Name: count, dtype: float64
count    17707.000000
mean         4.047495
std          6.845939
min          1.000000
25%          1.000000
50%          2.000000
75%          4.000000
max        100.000000
Name: count, dtype: float64


/tmp/ipykernel_286/2464061315.py:9: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  train_df = train_df.groupby('item_id', group_keys=False).apply(


# everything up to here should be in preprocessing

## want df with 4 columns: user_session_id, item_id, rating, split
## split = "train", "val", "test"

In [17]:
# create user session enconding and item encoding
user_session_id_to_idx = {user_session_id: idx for idx, user_session_id in enumerate(df['user_session_id'].unique())}
user_session_idx_to_id = {idx: user_session_id for user_session_id, idx in user_session_id_to_idx.items()}

item_id_to_idx = {item_id: idx for idx, item_id in enumerate(df['item_id'].unique())}
item_idx_to_id = {idx: item_id for item_id, idx in item_id_to_idx.items()}

# map values to idx using the above dicts
df["user_session_id"] = df["user_session_id"].map(user_session_id_to_idx)
df["item_id"] = df["item_id"].map(item_id_to_idx)

# get number of unique user_sessions and unique items
n_user_sessions = len(user_session_id_to_idx)
n_items = len(item_id_to_idx)

# instantiate dataset
dataset = UserSessionItemDataset(df[df.split == "train"], df[df.split == "val"], df[df.split == "test"], n_user_sessions, n_items)

In [18]:
l2s = [64., 256., 1024.,]

hyperparameter_selection(dataset, l2s, ndcg, k=100)

L2 64.0
Constructing G...
Density of G: 0.2602%
Inverting G...
EASE
ndcg @ 100: 0.12433571090756053 +- 0.0032288946888075288
AbsEASE
ndcg @ 100: 0.15731621433713747 +- 0.003960681190527962

L2 256.0
Constructing G...
Density of G: 0.2602%
Inverting G...
EASE
ndcg @ 100: 0.13621958212610266 +- 0.0035343284602157724
AbsEASE
ndcg @ 100: 0.15201580606250806 +- 0.003889969431115599

L2 1024.0
Constructing G...
Density of G: 0.2602%
Inverting G...
EASE
ndcg @ 100: 0.14202295547934587 +- 0.0036919280183094227
AbsEASE
ndcg @ 100: 0.14742060563336135 +- 0.0037845749129307815



best L2 for EASE is 1024., for AbsEASE is 64.

In [19]:
models = [("EASE", EASE, 1024.), ("AbsEASE", AbsEASE, 64.)]

run_test(models, dataset, ks=[10,20,50,100,200,500])

Split test
EASE
Constructing G...
Density of G: 0.2602%
Inverting G...
pos_inputs
recall_liked @ 10: 0.021957798436583284 +- 0.0006025498323860573
recall_disliked @ 10: 0.0037197163668640223 +- 0.0007111707248753739
ndcg @ 10: 0.04419161780892222 +- 0.0010531435119450429

recall_liked @ 20: 0.040074191573059234 +- 0.0009339801814301079
recall_disliked @ 20: 0.007239011656870498 +- 0.0009785943592240472
ndcg @ 20: 0.06341842548492532 +- 0.0012832016468521449

recall_liked @ 50: 0.0829221772596738 +- 0.0014458943619883944
recall_disliked @ 50: 0.016319088794178604 +- 0.0014999023298077477
ndcg @ 50: 0.10008262505917082 +- 0.0017115786003953318

recall_liked @ 100: 0.13350399559199042 +- 0.0019848950690388996
recall_disliked @ 100: 0.031201289995167254 +- 0.0020756187849005165
ndcg @ 100: 0.13653119173083766 +- 0.0021492128201289745

recall_liked @ 200: 0.20632883777280042 +- 0.0026118829415214795
recall_disliked @ 200: 0.05519759408318342 +- 0.002734028090183357
ndcg @ 200: 0.18188264688

TypeError: EASE.fit() missing 1 required positional argument: 'X'